<a href="https://colab.research.google.com/github/solcanalla/fiumark/blob/main/randomforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modelo: Random Forest#

Ensamble de tipo bagging.

##Inicialización##

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier

##Pre procesamiento##

In [2]:
!git clone https://github.com/solcanalla/fiumark.git
%cd fiumark
!git pull origin main
import preprocessing as pp

Cloning into 'fiumark'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 115 (delta 61), reused 43 (delta 17), pack-reused 0
Receiving objects: 100% (115/115), 31.76 KiB | 580.00 KiB/s, done.
Resolving deltas: 100% (61/61), done.
/content/fiumark
From https://github.com/solcanalla/fiumark
 * branch            main       -> FETCH_HEAD
Already up to date.


In [3]:
pd.options.display.max_columns = None

df = pp.get_dataset()
df = pp.decisiontree_preprocessing(df)
X_train, X_test, y_train, y_test = pp.get_train_test_data(df)

##Entrenamiento##

In [4]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': np.arange(50,100,10),
    'criterion': ['gini', 'entropy'],
    'max_depth': np.arange(1, 10),
    'max_features': ['sqrt', 'log2']
}

clf = RandomForestClassifier()

gscv = GridSearchCV(
    clf, params, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

In [5]:
print(f"Best score: {gscv.best_score_}")
print(f"Best params {gscv.best_params_}")

Best score: 0.8679314803453027
Best params {'criterion': 'entropy', 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 50}


##Evaluación del modelo##

In [7]:
y_pred = gscv.predict(X_test)

###Precision recall y F1-score###

In [8]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.95      0.87        95
           1       0.90      0.68      0.78        66

    accuracy                           0.84       161
   macro avg       0.86      0.81      0.82       161
weighted avg       0.85      0.84      0.83       161



###Accuracy###

In [9]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8385093167701864

###Auc-Roc###

In [10]:
from sklearn.metrics import roc_auc_score
auc_roc = roc_auc_score(y_test, y_pred)
auc_roc

0.8145933014354066

##Predicción##

In [11]:
X_holdout = pd.read_csv('https://drive.google.com/uc?export=download&id=1I980-_K9iOucJO26SG5_M8RELOQ5VB6A')
X = pp.decisiontree_preprocessing(X_holdout)
prediction = pd.DataFrame(gscv.predict(X),columns=['volveria'],dtype=int)
pred_formatted = pd.concat([X_holdout.id_usuario,prediction],axis=1)
pred_formatted.to_csv('randomforest.csv')
pred_formatted

,id_usuario,volveria
0,650,0
1,234,0
2,31,1
3,552,1
4,411,0
...,...,...
85,354,0
86,82,0
87,172,0
88,8,0
